In [34]:
import pandas as pd
import numpy as np

from Neurogram_short import *

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder 
from scipy.stats import randint


In [6]:
Tk().withdraw()  # keep the root window from appearing
dir_name = ('../datasets/')

path = '../datasets/rat7&8/day2'  # Port A
# path = '../datasets/rat7&8/day2' 
map_path = '../datasets/map_linear.csv'

# When using port A: channels=range(0,32,1) by default port B:range(32,64,1)
# Start and dur in samples
# feinstein: channels=[0]
time_start = time.time()
load_from_file = True  # Keep it always to true
downsample = 2  # Only when loading from raw - no need to use it
start = 0
dur = None

port = 'Port A'  # Select port A or B for different recordings
record = Recording.open_record(path, start=start, dur=dur,
                               load_from_file=load_from_file,
                               load_multiple_files=True,
                               downsample=downsample,
                               port=port,  # Select recording port
                               map_path=map_path,
                               verbose=0)

# Create directory to save figures
if not os.path.exists('%s/figures/' % (path)):
    os.makedirs('%s/figures/' % (path))
print("Time elapsed: {} seconds".format(time.time() - time_start))

#sys.exit()

../datasets/rat7&8/day2
Selecting file...
Loading from file C:/Users/airbl/OneDrive - University of Cambridge/Documents/Cambridge Work IIB/IIB Project/Code/code_Peter/datasets/rat7&8/day2/R7&8_5_minutes.pkl
0
stop: 3000001
10000.0
['14', '15', '16', '17', '18', '20']
                                ch_14      ch_15      ch_16   ch_17  \
time                                                                  
1970-01-01 00:00:00.000000 -23.400000 -27.299999  -2.340000  -5.070   
1970-01-01 00:00:00.000100 -19.305000 -12.675000  -4.290000  -1.950   
1970-01-01 00:00:00.000200 -14.040000  -4.290000  -0.195000   2.730   
1970-01-01 00:00:00.000300 -26.325001  -7.995000   4.485000  -0.780   
1970-01-01 00:00:00.000400 -13.845000  -0.195000  10.140000  -2.730   
...                               ...        ...        ...     ...   
1970-01-01 00:04:59.999600  -2.730000  -4.875000   9.165000   9.945   
1970-01-01 00:04:59.999700 -15.405000 -18.330000  -6.435000   0.585   
1970-01-01 00:04:59.99

In [3]:
test_file = '../datasets/rat7&8/day2/test8.csv'

In [18]:
windows_data = pd.read_csv(test_file, names=['start', 'end', 'category'])

In [15]:
print(record.recording)

                                ch_14      ch_15      ch_16   ch_17  \
time                                                                  
1970-01-01 00:00:00.000000 -23.400000 -27.299999  -2.340000  -5.070   
1970-01-01 00:00:00.000100 -19.305000 -12.675000  -4.290000  -1.950   
1970-01-01 00:00:00.000200 -14.040000  -4.290000  -0.195000   2.730   
1970-01-01 00:00:00.000300 -26.325001  -7.995000   4.485000  -0.780   
1970-01-01 00:00:00.000400 -13.845000  -0.195000  10.140000  -2.730   
...                               ...        ...        ...     ...   
1970-01-01 00:04:59.999600  -2.730000  -4.875000   9.165000   9.945   
1970-01-01 00:04:59.999700 -15.405000 -18.330000  -6.435000   0.585   
1970-01-01 00:04:59.999800 -28.860001 -31.980000 -19.500000 -29.055   
1970-01-01 00:04:59.999900 -31.785000 -34.904999 -22.424999 -30.615   
1970-01-01 00:05:00.000000 -38.220001 -32.759998 -28.275000 -36.660   

                                ch_18      ch_20   seconds  window  
time   

In [84]:
label_mapping = {
    's': 0,
    'n': 1,
    'm': 2,
    'e': 3
}

In [82]:
record.recording.columns

Index(['ch_14', 'ch_15', 'ch_16', 'ch_17', 'ch_18', 'ch_20', 'seconds',
       'window'],
      dtype='object')

In [40]:
neural_data = record.recording.filter(regex='^ch_')
print(neural_data)

                                ch_14      ch_15      ch_16   ch_17  \
time                                                                  
1970-01-01 00:00:00.000000 -23.400000 -27.299999  -2.340000  -5.070   
1970-01-01 00:00:00.000100 -19.305000 -12.675000  -4.290000  -1.950   
1970-01-01 00:00:00.000200 -14.040000  -4.290000  -0.195000   2.730   
1970-01-01 00:00:00.000300 -26.325001  -7.995000   4.485000  -0.780   
1970-01-01 00:00:00.000400 -13.845000  -0.195000  10.140000  -2.730   
...                               ...        ...        ...     ...   
1970-01-01 00:04:59.999600  -2.730000  -4.875000   9.165000   9.945   
1970-01-01 00:04:59.999700 -15.405000 -18.330000  -6.435000   0.585   
1970-01-01 00:04:59.999800 -28.860001 -31.980000 -19.500000 -29.055   
1970-01-01 00:04:59.999900 -31.785000 -34.904999 -22.424999 -30.615   
1970-01-01 00:05:00.000000 -38.220001 -32.759998 -28.275000 -36.660   

                                ch_18      ch_20  
time                     

In [89]:
channels = []
for col in neural_data.columns:
    if col.startswith('ch_'):
        channels.append(col)
print(channels)

['ch_14', 'ch_15', 'ch_16', 'ch_17', 'ch_18', 'ch_20']


In [41]:
neural_data = neural_data.reset_index(drop=False)
print(neural_data)

                              time      ch_14      ch_15      ch_16   ch_17  \
0       1970-01-01 00:00:00.000000 -23.400000 -27.299999  -2.340000  -5.070   
1       1970-01-01 00:00:00.000100 -19.305000 -12.675000  -4.290000  -1.950   
2       1970-01-01 00:00:00.000200 -14.040000  -4.290000  -0.195000   2.730   
3       1970-01-01 00:00:00.000300 -26.325001  -7.995000   4.485000  -0.780   
4       1970-01-01 00:00:00.000400 -13.845000  -0.195000  10.140000  -2.730   
...                            ...        ...        ...        ...     ...   
2999996 1970-01-01 00:04:59.999600  -2.730000  -4.875000   9.165000   9.945   
2999997 1970-01-01 00:04:59.999700 -15.405000 -18.330000  -6.435000   0.585   
2999998 1970-01-01 00:04:59.999800 -28.860001 -31.980000 -19.500000 -29.055   
2999999 1970-01-01 00:04:59.999900 -31.785000 -34.904999 -22.424999 -30.615   
3000000 1970-01-01 00:05:00.000000 -38.220001 -32.759998 -28.275000 -36.660   

             ch_18      ch_20  
0       -11.115000 

In [79]:
neural_data['time'][3000000]

AttributeError: 'Timestamp' object has no attribute 'as_unit'

In [77]:
a = pd.to_datetime(neural_data['time'])

TypeError: Cannot cast DatetimeArray to dtype float64

In [75]:
neural_data['time'].astype(int)

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [int32]

In [19]:
print(windows_data)

     start      end category
0  170.700  171.500        e
1  166.233  172.866        s
2  124.100  127.666        m
3  168.900  170.300        m
4  124.666  128.533        e
5  171.933  174.033        c
6  175.366  181.966        m
7  123.566  126.900        s
8  125.133  129.366        c
9   13.766   16.833        s


In [85]:
windows_data['category'] = windows_data['category'].map(label_mapping)

In [24]:
for i, row in windows_data.iterrows():
    start = pd.Timestamp(int(row[0]), unit='s')
    end = pd.Timestamp(int(row[1]), unit='s')
    cat = row[2]
    for ch in channels:
        window = windows_data[ch].loc[start:end]
        

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [39]:
record.recording.column_ch

AttributeError: 'DataFrame' object has no attribute 'column_ch'

In [36]:
feature_list = [
    'Mean' , 'Max' , 'Min' , 'MAV' , 'Var' , 'StD' , 'WL' , 'Energy_from_fft' , 'Kurtosis' , 'Skewness', 'Signal_Power' , 'Signal_Energy' , 'Min_max_difference' , 'Zero_crossing' , 'Slope_sign_changes', 'Wilson_Amplitude' , 'Root_Mean_Square' , 'V3'  , 'Log_detector' , 'DABS' , 'Maximum_fractal_length' ,'Myopulse_percentage_rate' , 'Mean_Frequency' 
]

In [ ]:
features_df = Feature_extraction_1(  feature_list, )

In [33]:
x, y = None, None

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

TypeError: Expected sequence or array-like, got <class 'NoneType'>

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

In [ ]:
y_pred = rf.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_pred, y_test)